# Ocultación de palabras futuras con atención casual

En  esta  sección,  se modifica el  mecanismo  de  autoatención  estándar  para  crear  un  mecanismo  de  atención  causal

La  atención  causal,  también  conocida  como  atención  enmascarada,  es  una  forma  especializada  de  autoatención.
 
Restringe  un  modelo  para  que  solo  considere  las  entradas  previas  y  actuales  de  una  secuencia  al  procesar  cualquier  token.  
Esto  contrasta  con  el  mecanismo  estándar  de  autoatención,  que  permite  acceder  a  toda  la  secuencia  de  entrada  a  la  vez.
 
En  consecuencia,  al  calcular  los  puntajes  de  atención,  el  mecanismo  de  atención  causal  garantiza  que  el  modelo  solo  
tenga  en  cuenta  los  tokens  que  aparecen  en  el  mismo  token  o  antes  que  este  en  la  secuencia.Para  lograr  esto  en  LLM  tipo  GPT,  para  cada  token  procesado,  enmascaramos  el  futuro tokens,  que  vienen  después  del  token  actual  en  el  texto  de  entrada

![Texto alternativo](./imgs/3.18.png)



## Aplicación  de  una  máscara  de  atención  causal

![Texto alternativo](./imgs/3.19.png)

Una  forma  de  obtener  la  matriz  de  ponderación  de  atención  enmascarada  en  la  atención  causal  es  aplicar  la  función  softmax  a  los  puntajes  de  atención,  poniendo  a  cero  los  elementos  por  encima  de  la  diagonal  y  normalizando  el  resultado de la matriz.



In [34]:
import torch
import torch.nn as nn

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     
    [0.55, 0.87, 0.66], # journey  (x^2)
    [0.57, 0.85, 0.64], # starts   (x^3)
    [0.22, 0.58, 0.33], # with     
    [0.77, 0.25, 0.10], # one      
    [0.05, 0.80, 0.55]] # step     
)

d_in = 3
d_out = 2

class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores =queries@keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [ ]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in=d_in, d_out=d_out) #A Reutilice  las  matrices  de  consulta  y  peso  clave  del  objeto  SelfAttention_v2  de  la  sección  anterior  para conveniencia
queries = sa_v2.W_query(inputs)                                   
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print(attn_weights)



tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


Se puede  implementar  el  paso  2  de  la  Figura   usando  la  función  tril  de  PyTorch  para  crear  una  máscara  donde  
los  valores  por  encima  de  la  diagonal  sean  cero.

In [36]:
context_length = attn_scores.shape[0]

mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

#Aplicar el enmascaramiento
simple_masked = attn_weights * mask_simple
print(simple_masked)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


El  tercer  paso  en  la  Figura es  renormalizar  los  pesos  de  atención  para  que  sumen  1  nuevamente  en  cada  fila.  Podemos  lograr  esto  dividiendo  cada  elemento  de  cada  fila  por  la  suma  de  cada  uno.


In [37]:
row_sums = simple_masked.sum(dim=1, keepdim=True)
masked_simple_norm = simple_masked / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


Cuando  aplicamos  una  máscara  y  luego  renormalizamos  los  pesos  de  atención,  podría  parecer  inicialmente  que  la  
información  de  los  tokens  futuros  (que  pretendemos  enmascarar)  aún  podría  influir  en  el  token  actual  porque  sus  valores  
son  parte  del  cálculo  de  softmax.

Sin  embargo,  la  idea  clave  es  que  cuando  renormalizamos  los  pesos  de  atención  después  del  enmascaramiento,  lo  que  
estamos  haciendo  esencialmente  es  recalcular  el  softmax  sobre  un  subconjunto  más  pequeño  (ya  que  las  posiciones  
enmascaradas  no  contribuyen  al  valor  softmax).

La  elegancia  matemática  de  softmax  es  que,  a  pesar  de  incluir  inicialmente  todas  las  posiciones  en  el  denominador,  
después  de  enmascarar  y  renormalizar,  el  efecto  de  las  posiciones  enmascaradas  se  anula;  no  contribuyen  al  puntaje  
de  softmax  de  ninguna  manera  significativa.

En  términos  más  simples,  después  del  enmascaramiento  y  la  renormalización,  la  distribución  de  los  pesos  de  atención  
es  como  si  se  hubiera  calculado  sólo  entre  las  posiciones  no  enmascaradas  desde  el  principio.
Esto  garantiza  que  no  haya  fugas  de  información  de  tokens  futuros  (o  de  otro  modo  enmascarados)  como  pretendíamos.

![Texto alternativo](./imgs/3.20.png)

Una  forma  más  eficiente  de  obtener  la  matriz  de  peso  de  atención  enmascarada  en  la  atención  causal  es  enmascarar  los  puntajes  de  atención  con  valores  infinitos  negativos  antes  de  aplicar  la  función  softmax.

La  función  softmax  convierte  sus  entradas  en  una  distribución  de  probabilidad.  Cuando  hay  valores  de  infinito  negativo  (∞)  
consecutivos,  la  función  softmax  los  trata  como  cero.probabilidad.  (Matemáticamente,  esto  se  debe  a  que  e ∞se  aproxima  a  0.)

Podemos  implementar  este  "truco"  de  enmascaramiento  más  eficiente  creando  una  máscara  con  1  arribala  diagonal  y  luego  reemplazar  estos  1  con  valores  negativos  de  infinito  ( inf)

In [38]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1) #matriz de unos en la diagonal superior
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


Ahora  podríamos  usar  los  pesos  de  atención  modificados  para  calcular  los  vectores  de  contexto  mediante  context_vec  =  
attn_weights  @  values

## Enmascaramiento de  pesos  de  atención  adicionales  con  abandono

La  deserción  en  el  aprendizaje  profundo  es  una  técnica  que  ignora  unidades  de  capa  oculta  seleccionadas  aleatoriamente  
durante  el  entrenamiento,  descartándolas.  Este  método  ayuda  a  prevenir  el  sobreajuste,  ya  que  garantiza  que  un  modelo  no  
dependa  excesivamente  de  ningún  conjunto  específico  de  unidades  de  capa  oculta.  Es  importante  destacar  que  la  deserción  
solo  se  utiliza  durante  el  entrenamiento.y  se  desactiva  posteriormente.

En  la  arquitectura  del  transformador,  incluidos  modelos  como  GPT,  la  pérdida  de  atención  en  el  mecanismo  se  aplica  
típicamente  en  dos  áreas  específicas:  después  de  calcular  los  puntajes  de  atención  o  después  de  aplicar  los  pesos  de  
atención  a  los  vectores  de  valores.

Se aplicará la  máscara  de  abandono  después  de  calcular  los  pesos  de  atención.

![Texto alternativo](./imgs/3.21.png)

Se utilizará  una  tasa  de  abandono  del  50  %,  lo  que  implica  enmascarar  la  mitad  de  los  pesos  de  atención. 
Se aplica primero  la  implementación  de  abandono  de  PyTorch  a  un  tensor  de  6×6.

In [39]:
torch.manual_seed(123)

dropout = nn.Dropout(0.5)
example = torch.ones(6, 6)

print(example)
print(dropout(example))

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


La  mitad  de  los  elementos  de  la  matriz  se  establecen  aleatoriamente  en  cero.  Para  compensar  la  reducción  de  elementos  activos,  los  valores  de  los  elementos  restantes  de  la  matriz  se  escalan  por  un  factor  de  1/0,5  =  2.  Este  escalamiento  es  crucial  para  mantener  el  equilibrio  general  de  las  ponderaciones  de  atención,  garantizando  que  la  influencia  promedio  del  mecanismo  de  atención  se  mantenga  constante  durante  las  fases  de  entrenamiento  e  inferencia.

## Implementación  de  una  clase  de  atención  causal  compacta

Incorporación de las modificaciones  de  atención  causal  y  abandono  en  la  clase  de  Python  SelfAttention  que se han desarrollado.  Esta  clase  servirá  como  plantilla  para  desarrollar  la  atención  multicabeza  en  la  siguiente  sección,  que  es  la  última  clase  de  atención  que  implementamos  en  este  capítulo.

Antes  de  comenzar,  una  cosa  más  es  asegurarnos  de  que  el  código  pueda  manejar  lotes  que  constan  
de  más  de  una  entrada  para  que  la  clase  CausalAttention  admita  las  salidas  por  lotes  producidas  por  el  cargador  
de  datos.

In [40]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)  
batch

torch.Size([2, 6, 3])


tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])

La  siguiente  clase  CausalAttention  es  similar  a  la  clase  SelfAttention,  excepto  que  ahora  se agregan  los  componentes  de  abandono  y  máscara  causal.

In [41]:
class CasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout,  qkv_bias=False):
        super().__init__()
        self.out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = nn.Dropout(dropout)  #Capa de abandono

        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length),diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape                            
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.transpose(1, 2)              #transponer las dimensiones 1 y 2,manteniendo la dimension del lote en la primera dimension (0) 
        attn_scores.masked_fill_(                                 #las operaciones con guion dinal sen realiza en el lugar lo que evita copias de meoria inecesarias
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf) 
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec


Ahora  se ha  añadido  una  llamada  a  self.register_buffer()  en  el método  __init__ .  El  uso  de  register_buffer  en  PyTorch  no  es  estrictamente  necesario  para  todos  los  casos  de  uso,  pero  ofrece  varias  ventajas.  Por  ejemplo,  al  usar  la  clase  CausalAttention  en  nuestro  LLM,  los  búferes  se  mueven  automáticamente  al  dispositivo  correspondiente  (CPU  o  GPU)  junto  con  nuestro  modelo,  lo  cual  será  relevante  al  entrenar  el  LLM  . No es necesario aegurarse que los tensores estén en el mismo dispositivo que los parámetros del modelo. Evita discrepacias.

In [42]:
torch.manual_seed(123)

context_length = batch.shape[1]
ca = CasualAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


![Texto alternativo](./imgs/3.22.png)

[Ampliación de la atención de una sola cabeza a la atención de múltiples cabezas](./6_atencion_multicabecera.ipynb)